In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv('GA_2_dataset.csv')

In [6]:
object_cols = df.select_dtypes(include='object').columns
print(object_cols.to_list())

['Gender', 'Location', 'GameGenre', 'GameDifficulty', 'EngagementLevel']


In [7]:
cond1 = (
    (df['Gender'] == 'Male')&
    (df['Location'] == 'Europe')&
    (df['InGamePurchases']) == 1  
)
count = df[cond1].shape[0]
print(count)

299


In [9]:
cond2 = (df['Age']<18) & (df['PlayTimeHours']>10)
countt = df[cond2].shape[0]
print(countt)

453


Split the dataset into train dataset and test dataset in the following manner.

Use sklearn train_test_split function to split the data.

Use only 20% data as test_set and keep random_state = 42

Which category has the least value counts in y_train?

In [ ]:
from sklearn.model_selection import train_test_split
X = df.drop('EngagementLevel', axis=1) #axis = 0 means drop a row
y = df['EngagementLevel']
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)
print(y_train.value_counts())
least_comm = y_train.value_counts().idxmin()
print(least_comm)

EngagementLevel
Medium    3983
Low       2021
High      1996
Name: count, dtype: int64
High



Create feature matrix(X) and label vector(y) using following instructions:

"EngagementLevel" is the target column(y).

All the columns except the target column are in feature matrix(X).

How many total null values were present in the whole dataset ?

In [14]:
X = df.drop('EngagementLevel', axis=1)
y = df['EngagementLevel']

# Count total null (missing) values in the entire dataset
total_null = df.isnull().sum().sum()
print("Total null values in the dataset:", total_null)

Total null values in the dataset: 3337


In [ ]:
# Step 1: Split into train and test sets
X = df.drop('EngagementLevel', axis=1)
y = df['EngagementLevel']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)



# ---- AGE ----
# Compute mean from training set excluding missing values
age_mean = X_train['Age'].dropna().astype(float).mean()
# Replace missing/Unknown with mean in both sets
X_train['Age'] = X_train['Age'].fillna(age_mean)
X_test['Age'] = X_test['Age'].fillna(age_mean)

# ---- LOCATION ----
X_train['Location'] = X_train['Location'].fillna('Other')
X_test['Location'] = X_test['Location'].fillna('Other')

# ---- GAMEDIFFICULTY ----
game_diff_mode = X_train['GameDifficulty'].dropna().mode()[0]
X_train['GameDifficulty'] = X_train['GameDifficulty'].fillna(game_diff_mode)
X_test['GameDifficulty'] = X_test['GameDifficulty'].fillna(game_diff_mode)

# ---- INGAMEPURCHASES ----
X_train['InGamePurchases'] = X_train['InGamePurchases'].fillna('0')
X_test['InGamePurchases'] = X_test['InGamePurchases'].fillna('0')

# ---------------------------
# Step 3: Calculate required answer
# ---------------------------
sum_age_test = X_test['Age'].astype(float).sum().round(2)
print("Sum of imputed 'Age' column in test dataset:", sum_age_test)

Sum of imputed 'Age' column in test dataset: 63585.24


In [17]:
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer

transformer_list = [
    ("mean_imp",SimpleImputer(strategy="mean"),["Age"]),
    ("const_imp",SimpleImputer(strategy="constant",fill_value="Other"),["Location"]),
    ("mode_imp",SimpleImputer(strategy="most_frequent"),["GameDifficulty"]),
    ("const_2_imp",SimpleImputer(strategy="constant",fill_value=0),["InGamePurchases"])

]

ct = ColumnTransformer(transformers = transformer_list, remainder='passthrough',verbose_feature_names_out=False).set_output(transform="pandas")
ct

,transformers,"[('mean_imp', ...), ('const_imp', ...), ...]"
,remainder,'passthrough'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,False
,force_int_remainder_cols,'deprecated'
,missing_values,nan
,strategy,'mean'
,fill_value,None


In [18]:
X_train_impute = ct.fit_transform(X_train)
X_test_impute = ct.transform(X_test)
X_test_impute["Age"].sum()

np.float64(63585.239130434784)


Apply preprocessing on features of train and test datasets.

Drop the "PlayerID" Column before the preprocessing steps.
Before applying any preprocessing there should not be any missing or unknown values present in the train and test dataset.
Learn transformers' parameters using training set only and then transform train & test sets using them.

For Categorical Features

Ordinal Features

Ordinally Encode "GameDifficulty"

GameDifficulty   	Order
Easy	0
Medium	1
Hard	2


Nominal Features
One-Hot Encode 'Gender', 'Location', 'GameGenre' features and keep drop_first = True.

Scaling Features

Scale all the features (transformed categorical and numerical) of the feature matrix using the StandardScaler

Calculate the sum of all the values present in first five rows of transformed test feature matrix ? (upto 2 digits afer the decimal)

In [24]:
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler
X_train_impute.drop('PlayerID', axis=1, inplace=True)
X_test_impute.drop('PlayerID', axis=1, inplace=True)
trf = [
    ("OHE",OneHotEncoder(sparse_output=False ,drop = 'first'),['Gender', 'Location', 'GameGenre']),
    ("OE",OrdinalEncoder(categories=[['Easy','Medium','Hard']]),["GameDifficulty"])
]

ct2 = ColumnTransformer(trf, remainder='passthrough',verbose_feature_names_out=False).set_output(transform="pandas")

ct2

KeyError: "['PlayerID'] not found in axis"